# Projet de Machine Learning

In [1]:
from tools import *
import math as math
import copy
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def gen_arti(centerx=1,centery=1,sigma=0.1,nbex=10,data_type=0,eps=0.02):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param eps: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
         #melange de 2 gaussiennes
         xpos=np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//2)
         xneg=np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//2)
         data=np.vstack((xpos,xneg))
         y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,eps,nbex)
    data[:,1]+=np.random.normal(0,eps,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

    

In [79]:
def f_K(x, e, sigma=1):
    return math.exp(-pow(np.linalg.norm(x-e), 2)/(2*pow(sigma,2)))

def HA(a, S, dataset, y):
    N = dataset.shape[0]
    Kql= np.zeros((N,N))
    Kal = np.zeros((N,N))
    f = np.zeros((0, 0))
    
    print "Calcul des fl..."
    for l,x in enumerate(dataset):
        fl = a[l]*f_K(x, x)
        for j,e in enumerate(S):
            fl += a[j]*K[l,j]
        f = np.append(f, fl)
        
    print "Done"

    for i,x in enumerate(S):
        for j,e in enumerate(S):
            Kql[i,j] = f_K(x,e)
            
    Kql = np.asmatrix(np.asarray(Kql))
    f = np.asmatrix(f.reshape((f.shape[0], 1)))
    y = np.asmatrix(y)

    
    print "Calcul de H(xl)...", -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a
    return -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a

def H(x, S, a_prec, dataset, y):

    a = a_optim(x, S, a_prec) 
    return HA(a, S, dataset, y)

def a_optim(x, S, a_prec):

    f = K*a_prec
    p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
    x = np.asarray(p*(1-p))
    W = np.asmatrix(np.zeros((x.shape[0], x.shape[0])))
    
    for i,element in enumerate(x):
        W[i,i] = element
        
    z = K*a_prec+np.linalg.inv(W)*(y-p)
    return np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 

def ivm(dataset, y):

    N = dataset.shape[0]
    a = np.zeros((N,1))
    S = []
    
    lamda = 3
    
    Hk = 1
    Hk_1 = 0

    while abs(Hk-Hk_1)/abs(Hk) > 0.01:
        h = np.zeros((N, 1))
        S_b = []
        for i in range(N):
            for e in S:
                S_b.append(e)
            S_b.append(dataset[i])
            h[i] = H(dataset[i], S_b, a, dataset, y)
            print h[i]
        
        print "Done"
        print h
        print "On recupere le plus petit H(xl)"
        xl_opt = np.argmin(h)
        Hk_1 = Hk
        Hk = h[xl_opt]
        print "Le plus petit est ", Hk, " il est à l'indice : ", xl_opt
        
        print S
        print "On l'ajoute a S..."
        print dataset[xl_opt]
        S.append(dataset[xl_opt])
        print "OK"
        print S
        print "On l'enleve a dataset... Taille avant:", dataset.shape
        print "data avant:", dataset
        data = np.zeros((dataset.shape[0]-1,2))
        for i, e in enumerate(dataset):
            if i == xl_opt:
                print "on degage l'element", e
                pass
            else:
                print "on garde l'element", e
                data[i] = e
        print "OK, taille apres:", data.shape
        print "data:", data
        dataset = data
        print "final dataset:", dataset
        print "final shape:", dataset.shape
        # update de a
        print "Updating a..."
        a_prec = a
        f = K*a_prec
        p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
        x = np.asarray(p*(1-p))
        W = np.asmatrix(np.zeros((x.shape[0], x.shape[0])))

        for i,element in enumerate(x):
            W[i,i] = element
        z = K*a_prec+np.linalg.inv(W)*(y-p)
        a = np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 
        print "OK"
        print abs(Hk-Hk_1)/abs(Hk)
        
    print "S", S


In [80]:
data, y = gen_arti()
data = np.asarray(data)
print data
N = data.shape[0]
K = np.asmatrix(np.zeros((N,N)))
lamda=1
for i,x in enumerate(data):
    for j,e in enumerate(data):
        K[i,j] = f_K(x,e)

y = y.reshape((y.shape[0], 1))
ivm(data, y)

[[ 1.6366156   0.8133874 ]
 [ 1.36118745  0.90840529]
 [ 1.47732154  1.19550378]
 [ 1.12656992  0.59636843]
 [-1.37400935 -1.50722917]
 [-0.9847511  -0.70415662]
 [-1.40459848 -1.19973321]
 [-1.08366089 -0.8234739 ]
 [-0.79410743 -0.59695867]
 [ 1.13390127  0.96531286]]
Calcul des fl...
Done
Calcul de H(xl)... [[-52.21196067]]
[-52.21196067]
Calcul des fl...
Done
Calcul de H(xl)... [[-62.21381886]]
[-62.21381886]
Calcul des fl...
Done
Calcul de H(xl)... [[-72.10595347]]
[-72.10595347]
Calcul des fl...
Done
Calcul de H(xl)... [[-82.57223335]]
[-82.57223335]
Calcul des fl...
Done
Calcul de H(xl)... [[-114.16648465]]
[-114.16648465]
Calcul des fl...
Done
Calcul de H(xl)... [[-145.21771631]]
[-145.21771631]
Calcul des fl...
Done
Calcul de H(xl)... [[-176.33671103]]
[-176.33671103]
Calcul des fl...
Done
Calcul de H(xl)... [[-207.00968647]]
[-207.00968647]
Calcul des fl...
Done
Calcul de H(xl)... [[-237.37075097]]
[-237.37075097]
Calcul des fl...
Done
Calcul de H(xl)... [[-247.45084752]]
[-2

ValueError: shapes (1,10) and (9,1) not aligned: 10 (dim 1) != 9 (dim 0)

In [81]:
for l, x in enumerate(data):
    print l

 0
1
2
3
4
5
6
7
8
9
